좌측 하단에 터미널 클릭 후 unzip cat_and_dogs.zip  -d data 로 압축 풀기

In [16]:
import glob, cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

def load_image(file):
    """
    경로 입력시 이미지로 불러와서 224,224,3 이미지로 변환
    """
    img = cv2.imread(file)
    img = img[...,::-1] # 마지막 차원이 RGB 채널을 의미
    img = cv2.resize(img, (224, 224))
    return img

def get_item_fun(df, index, batch_size):
    st = index*batch_size
    en = (index+1)*batch_size

    selected_df = df.iloc[st:en,:]

    imgs = []
    for path in selected_df['path']:
        img = load_image(path)
        imgs.append(img)
    imgs = np.array(imgs)

    x = imgs
    y = selected_df["label"].values
    return x, y

class MyGenerator(tf.keras.utils.Sequence):
    def __init__(
            self, df, batch_size,
            load_image, get_item_fun):
        self.df = df
        self.batch_size = batch_size
        self.load_image = load_image
        self.get_item_fun = get_item_fun
    def __len__(self):
        return int(np.ceil(self.df.shape[0]/self.batch_size))
    def __getitem__(self, index):
        x, y = self.get_item_fun(
            self.df, index, self.batch_size)
        return x, y
    def on_epoch_end(self):
        self.df = self.df.sample(frac=1)

In [17]:
file_dir = "/content/data"
cats = glob.glob(f"{file_dir}/Cat/*")
dogs = glob.glob(f"{file_dir}/Dog/*")
len(cats), len(dogs), len(cats) + len(dogs)

cats_df = pd.DataFrame({"label":1, "path":cats})
dogs_df = pd.DataFrame({"label":0, "path":dogs})
df = pd.concat([cats_df, dogs_df], ignore_index=True)
df=df.sample(frac=1, random_state = 42).reset_index(drop=True)

In [18]:
df

label                         path
0          1   /content/data/Cat/6718.jpg
1          0   /content/data/Dog/2520.jpg
2          1   /content/data/Cat/8566.jpg
3          1    /content/data/Cat/567.jpg
4          1  /content/data/Cat/11975.jpg
...      ...                          ...
23405      0   /content/data/Dog/2408.jpg
23406      0   /content/data/Dog/1307.jpg
23407      1   /content/data/Cat/5628.jpg
23408      1  /content/data/Cat/10793.jpg
23409      0   /content/data/Dog/2680.jpg

[23410 rows x 2 columns]

In [19]:
train_df, valid_df = train_test_split(
    df, random_state=42,
    test_size=0.2, stratify=df["label"])
test_df, valid_df = train_test_split(
    valid_df, random_state=42,
    test_size=0.5, stratify=valid_df["label"])
batch_size = 32
train_gen = MyGenerator(
    train_df, batch_size, load_image, get_item_fun)
valid_gen = MyGenerator(
    valid_df, batch_size, load_image, get_item_fun)

In [20]:
inputs = tf.keras.layers.Input(
    shape = (224,224,3))
x = tf.keras.layers.Conv2D(
    64, kernel_size=3, padding="same")(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(
    128, kernel_size=3, padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(
    256, kernel_size=3, padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(
    512, kernel_size=3, padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 28, 28, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 28, 28, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,620,609 (6.18 MB)

 Trainable params: 1,618,689 (6.17 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [21]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.001)
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.BinaryAccuracy()]
model.compile(
    optimizer=optimizer,
    loss = loss,
    metrics = metrics)

history = model.fit(
    train_gen, validation_data = valid_gen,
    epochs = 10)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 28, 28, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 28, 28, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,620,609 (6.18 MB)

 Trainable params: 1,618,689 (6.17 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


586/586 ━━━━━━━━━━━━━━━━━━━━ 122s 172ms/step - binary_accuracy: 0.5983 - loss: 0.7280 - val_binary_accuracy: 0.6873 - val_loss: 0.5912
Epoch 2/10
586/586 ━━━━━━━━━━━━━━━━━━━━ 118s 161ms/step - binary_accuracy: 0.6733 - loss: 0.6041 - val_binary_accuracy: 0.7240 - val_loss: 0.5540
Epoch 3/10
586/586 ━━━━━━━━━━━━━━━━━━━━ 95s 162ms/step - binary_accuracy: 0.7134 - loss: 0.5554 - val_binary_accuracy: 0.6437 - val_loss: 0.6653
Epoch 4/10
586/586 ━━━━━━━━━━━━━━━━━━━━ 100s 171ms/step - binary_accuracy: 0.7414 - loss: 0.5207 - val_binary_accuracy: 0.7437 - val_loss: 0.5392
Epoch 5/10
586/586 ━━━━━━━━━━━━━━━━━━━━ 101s 172ms/step - binary_accuracy: 0.7687 - loss: 0.4906 - val_binary_accuracy: 0.7185 - val_loss: 0.5598
Epoch 6/10
586/586 ━━━━━━━━━━━━━━━━━━━━ 137s 164ms/step - binary_accuracy: 0.7822 - loss: 0.4630 - val_binary_accuracy: 0.7010 - val_loss: 0.5560
Epoch 7/10
586/586 ━━━━━━━━━━━━━━━━━━━━ 95s 162ms/step - binary_accuracy: 0.8108 - loss: 0.4171 - val_binary_accuracy: 0.8287 - val_loss

In [29]:
import matplotlib.pyplot as plt
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

In [43]:
# 전이학습: 기존에 잘 만들어져 있는 모델을 가져와서, 그 모델을 활용하여 내 모델로 만드는 것을 의미
# 작은 데이터에서도 금방 성능을 높일 수 있어서 많이 활용

# 모바일 넷 에서 7, 7, 1280 의 변수만 뽑아서 변수를 활용해서 내 모델을 만듬
backbone = tf.keras.applications.MobileNetV2(
    input_shape = (224, 224, 3), include_top = False
)
backbone.trainable = False # 모델이 학습과정에서 weight가 갱신되지 않게 바뀜

In [44]:
inputs = tf.keras.layers.Input(shape=(224,224,3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = backbone(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

In [45]:
model = tf.keras.Model(inputs, outputs)

In [46]:
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_2 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_2 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.001)
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.BinaryAccuracy()]
model.compile(
    optimizer=optimizer,
    loss = loss,
    metrics = metrics)

try:
  history = model.fit(
      train_gen, validation_data = valid_gen,
      epochs = 10,
      workers=4,
      use_multiprocessing=True,
      )
except:
  history = model.fit(
      train_gen, validation_data = valid_gen,
      epochs = 10,
      )

Epoch 1/10
435/586 ━━━━━━━━━━━━━━━━━━━━ 14s 95ms/step - binary_accuracy: 0.9945 - loss: 0.0184